In [1]:
import tensorflow as tf
import numpy as np
from math import sqrt
from random import randint, shuffle
from tensorflow.keras.layers import Dense, Flatten
import copy

In [2]:
model = tf.keras.Sequential()

model.add(Flatten(input_shape=(9,9)))
model.add(Dense(81*2, activation=tf.math.sigmoid))
model.add(Dense(81*4, activation=tf.math.sigmoid))
model.add(Dense(27*8, activation=tf.math.sigmoid))
model.add(Dense(27*16, activation=tf.math.sigmoid))
model.add(Dense(27*16, activation=tf.math.sigmoid))
model.add(Dense(27*8, activation=tf.math.sigmoid))
model.add(Dense(27*4, activation=tf.math.sigmoid))
model.add(Dense(27*2, activation=tf.math.sigmoid))
model.add(Dense(27*2, activation=tf.math.sigmoid))
model.add(Dense(9+9+9, activation=tf.nn.relu))

print(model.summary())

model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.MeanSquaredError(), metrics=[])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 81)                0         
                                                                 
 dense (Dense)               (None, 162)               13284     
                                                                 
 dense_1 (Dense)             (None, 324)               52812     
                                                                 
 dense_2 (Dense)             (None, 216)               70200     
                                                                 
 dense_3 (Dense)             (None, 432)               93744     
                                                                 
 dense_4 (Dense)             (None, 432)               187056    
                                                                 
 dense_5 (Dense)             (None, 216)               9

2022-03-21 15:48:32.387466: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [39]:

def init(n):
    return [[0 for j in range(n)] for i in range(n)]


def affichage_grille(grille):
    n = len(grille)
    r = int(sqrt(n))
    for i in range(n):
        for j in range(n):
            print(grille[i][j], end="")
            if (j%r==r-1):
                print(" ", end="")
        print()
        if (i%r==r-1):
            print()


def checkGrid(grille):
    n = len(grille)
    for i in range(n):
        for j in range(n):
            if grille[i][j]==0:
                return False
    return True


listeNombres = [1, 2, 3, 4, 5, 6, 7, 8, 9]


def fillGrid(grille):
    n = len(grille)
    r = int(sqrt(n))
    #Find next empty cell
    for i in range(0,n**2):
        row = i//n
        col = i%n
        if grille[row][col] == 0:
            shuffle(listeNombres)
            for value in listeNombres:
                #Check that this value has not already be used on this row
                if not(value in grille[row]):
                #Check that this value has not already be used on this column
                    if not value in [grille[k][col] for k in range(n)]:
                        #Identify which of the 9 squares we are working on
                        isquare = (row//r)*r
                        jsquare = (col//r)*r
                        square=[grille[k][jsquare : jsquare+r] for k in range(isquare, isquare+r)]
                        #Check that this value has not already be used on this 3x3 square
                        if not value in [el for ligne in square for el in ligne]:
                            grille[row][col] = value
                            if checkGrid(grille):
                                return True
                            else:
                                if fillGrid(grille):
                                    return True
            break
    grille[row][col] = 0


def deconstruit_un_coup(grille):
    n = len(grille)
    while True:
        i, j = randint(0,n-1), randint(0,n-1)
        if (grille[i][j] != 0):
            break
    move = grille[i][j]
    grille[i][j] = 0
    resultat = [0 for i in range(3*n)]
    resultat[i-1] = 1
    resultat[n+j-1] = 1
    resultat[2*n+move-1] = 1
    return resultat


def joue(grille, i, j, move):
    grille[i][j] = move


def creation_donnees(repetition, profondeurPartie, n):
    train_grilles, train_move = [], []
    for i in range(repetition):
        partie = init(n)
        fillGrid(partie)
        for j in range(min(profondeurPartie, n**2)):
            move = deconstruit_un_coup(partie)
            train_grilles.append(copy.deepcopy(partie))
            train_move.append(move)
    return train_grilles, train_move


def melange_jeu(grille, profondeur):
    n = len(grille)
    for i in range(min(profondeur, n**2)):
        deconstruit_un_coup(grille)


def train():
    #Entrainement
    for i in range(1):
        print("n°",i)
        inData, outData = creation_donnees(100, 65, 9)
        model.fit(inData, outData, epochs=20)


def simulation(model, melange):
    partie = init(9)
    fillGrid(partie)
    melange_jeu(partie, melange)
    affichage_grille(partie)
    print()
    while (not checkGrid(partie)):
        prediction = model.predict([partie])
        i = 1 + max(range(len(prediction[0][0:9])), key=prediction[0][0:9].__getitem__)
        j = 1 + max(range(len(prediction[0][9:18])), key=prediction[0][9:18].__getitem__)
        move = 1 + max(range(len(prediction[0][18:27])), key=prediction[0][18:27].__getitem__)
        joue(partie, i, j, move)
        print(i, j, move)
    



In [6]:
train()

n° 0
Epoch 1/20
204/204 [==============================] - 1s 4ms/step - loss: 0.1136
Epoch 2/20
204/204 [==============================] - 1s 3ms/step - loss: 0.1059
Epoch 3/20
204/204 [==============================] - 1s 3ms/step - loss: 0.1059
Epoch 4/20
204/204 [==============================] - 1s 4ms/step - loss: 0.1059
Epoch 5/20
204/204 [==============================] - 1s 3ms/step - loss: 0.1059
Epoch 6/20
204/204 [==============================] - 1s 3ms/step - loss: 0.1059
Epoch 7/20
204/204 [==============================] - 1s 4ms/step - loss: 0.1059
Epoch 8/20
204/204 [==============================] - 1s 3ms/step - loss: 0.1059
Epoch 9/20
204/204 [==============================] - 1s 4ms/step - loss: 0.1059
Epoch 10/20
204/204 [==============================] - 1s 4ms/step - loss: 0.1059
Epoch 11/20
204/204 [==============================] - 1s 3ms/step - loss: 0.1060
Epoch 12/20
204/204 [==============================] - 1s 3ms/step - loss: 0.1059
Epoch 13/20
204/204 

In [40]:
simulation(model, 65)

100 000 000 
000 000 008 
800 040 001 

380 400 050 
000 060 000 
400 801 000 

000 000 000 
000 090 005 
000 000 100 


4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4 5
4 4

KeyboardInterrupt: 